<a href="https://colab.research.google.com/github/pooja614/ML_DL_projects/blob/main/SQL_IPL_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXPLORATORY DATA ANLAYSIS USING SQL

## About the data
* This [data](https://www.kaggle.com/datasets/harsha547/ipldatabase) is the database of Indian Premier League from season 2008-2016.  
* The data has record of 577 matches, 469 players with detailed match events and player information

* The schema of the data is as follows.

<img src='https://i.imgur.com/327NVKH.png'>

https://i.imgur.com/327NVKH.png

### Download and Unzip the data

In [ ]:
# Download dataset
# import os
# os.environ['KAGGLE_USERNAME'] = 'YY'
# os.environ['KAGGLE_KEY'] = 'XX'
# !kaggle datasets download -d harsha547/ipldatabase

  0% 0.00/4.29M [00:00<?, ?B/s]
100% 4.29M/4.29M [00:00<00:00, 161MB/s]


In [ ]:
# unzip the file
# import zipfile
# data_path ='/content/drive/MyDrive/ETE_projects/IPL_SQL/data/'
# zip_path = '/content/ipldatabase.zip'
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#   zip_ref.extractall(data_path)

In [ ]:
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
db_path = '/content/drive/MyDrive/ETE_projects/IPL_SQL/data/database.sqlite'
conn = sqlite3.connect(db_path)
c = conn.cursor()

### View tables

In [ ]:
# sql = "select * from sqlite_schema;"
sql = "SELECT name FROM sqlite_master WHERE type='table';"
dd = pd.read_sql(sql, conn)
dd

,name
0,Player
1,Extra_Runs
2,Batsman_Scored
3,Batting_Style
4,Bowling_Style
5,Country
6,Season
7,City
8,Outcome
9,Win_By


In [ ]:
# sql1 = "SELECT * from sqlite_master;"

In [ ]:
queries = [
    "pragma table_info('Player');",
"pragma table_info('Match');",
"pragma table_info('Team');"
]
for q in queries:
  print("---------------------------------------------------")
  print(q)
  print("---------------------------------------------------")
  res = pd.read_sql(q, conn)
  print(res)

---------------------------------------------------
pragma table_info('Player');
---------------------------------------------------
   cid           name          type  notnull dflt_value  pk
0    0      Player_Id       integer        1       None   1
1    1    Player_Name  varchar(400)        1       None   0
2    2            DOB      datetime        0       None   0
3    3   Batting_hand       integer        1       None   0
4    4  Bowling_skill       integer        0       None   0
5    5   Country_Name       integer        1       None   0
---------------------------------------------------
pragma table_info('Match');
---------------------------------------------------
    cid              name      type  notnull dflt_value  pk
0     0          Match_Id   integer        1       None   1
1     1            Team_1   integer        1       None   0
2     2            Team_2   integer        1       None   0
3     3        Match_Date  datetime        1       None   0
4     4        

In [ ]:
!pip install simple-colors

In [ ]:
from simple_colors import *

## Questions and necessary functions

In [ ]:
# match and season related
questions= ["Select total matches, matches won and lost and won percentage fo different teams.",
            "Top 10 run scorers overall and total number of sixes and fours by them",
            "Highest run scorers by batsmen season wise",
            "Top 10 wicket takers overall with their country",
            "Display players with top 5 strike rates ",
            "Man of Series, Orange Cap, Purple Cap winners",
            "Players with most centuries along with their season",
            "Types of wickets and total counts",
            "Player with highest number of sixes",
            "Players with most number of ducks",
            "Select the best economy bowlers in IPL",
            "Indian players below 21 years of age in the year 2016",
            ]

In [ ]:
def print_question(num):
  print(blue(questions[num], 'bold'))

def query_execute(q):
   return pd.read_sql(q, conn)

def describe_table(col):
  q = "pragma table_info({});".format(col)
  return pd.read_sql(q, conn)

def display_table(col):
  q = "select * from {} limit 3;".format(col)
  return pd.read_sql(q, conn)


### Select total matches, matches won and lost and won percentage fo different teams.

In [ ]:
q1 = """
select t.team_name as team_name, d.total_matches as total_match, d.win as matches_won, (d.total_matches - d.win) as matches_lost,100 * d.win/d.total_matches as win_percentage from
(
select a.team1 as teams,a.winner as win, tm_2_cnt+tm_1_cnt as total_matches from
(
select m.team_1 as team1,count(m.match_winner) as winner, count(m.match_id) as tm_1_cnt from match m
group by m.team_1
) a
JOIN (
select n.team_2 as team2, count(n.match_id) as tm_2_cnt from match n
group by n.team_2
) b
ON a.team1 = b.team2
order by total_matches
) d
join team t
on t.team_id = d.teams;


"""
print_question(0)
query_execute(q1)


Select total matches, matches won and lost and won percentage fo different teams.


,team_name,total_match,matches_won,matches_lost,win_percentage
0,Kochi Tuskers Kerala,14,7,7,50
1,Rising Pune Supergiants,14,7,7,50
2,Gujarat Lions,16,9,7,56
3,Pune Warriors,46,23,23,50
4,Sunrisers Hyderabad,62,30,32,48
5,Deccan Chargers,75,39,36,52
6,Rajasthan Royals,118,52,66,44
7,Chennai Super Kings,131,74,57,56
8,Kolkata Knight Riders,132,62,70,46
9,Delhi Daredevils,133,68,65,51


### Top 10 run scorers overall and total number of sixes and fours by them

In [ ]:
q  = """
select p.player_name as player_name, c.runs as runs_scored, c.sixes as no_of_sixes, c.fours as no_of_fours
from
(
select a.striker as striker, sum(b.runs_scored) as runs,
COUNT(case when b.runs_scored=6 then 1 else null end ) as sixes,
COUNT(case when b.runs_scored=4 then 1 else null end ) as fours from
ball_by_ball a
join batsman_scored b
on a.match_id = b.match_id
and a.over_id= b.over_id
and a.ball_id = b.ball_id
and a.innings_no = b.innings_no
group by a.striker
order by 2 desc
limit 10
) c
join player p
on c.striker = p.player_id;


"""
print_question(1)
query_execute(q)

Top 10 run scorers overall and total number of sixes and fours by them


,player_name,runs_scored,no_of_sixes,no_of_fours
0,SK Raina,4106,161,360
1,V Kohli,4105,148,360
2,RG Sharma,3874,164,323
3,G Gambhir,3634,51,422
4,CH Gayle,3447,252,281
5,RV Uthappa,3390,104,341
6,DA Warner,3373,134,338
7,AB de Villiers,3270,142,275
8,MS Dhoni,3270,140,236
9,S Dhawan,3082,62,348


### Highest run scorers by batsmen season wise

In [ ]:
q = """
select c.season, p.player_name as player, max(c.runs_scored) as runs
   from (
    select m.season_id as season, a.striker as striker, sum(b.runs_scored) as runs_scored
    from ball_by_ball a
    join batsman_scored b
    on a.match_id = b.match_id
    and a.over_id = b.over_id
    and a.ball_id  = b.ball_id
    and a.innings_no = b.innings_no
    join match m
    on a.match_id = m.match_id
    group by 1,2
    order by 1 asc, 3 desc
    ) c
    join player p
    on p.player_id = c.striker
    group by 1
    order by 1
"""
print_question(2)
query_execute(q)

Highest run scorers by batsmen season wise


,season,player,runs
0,1,SE Marsh,616
1,2,ML Hayden,572
2,3,SR Tendulkar,617
3,4,CH Gayle,604
4,5,CH Gayle,733
5,6,MEK Hussey,733
6,7,RV Uthappa,660
7,8,DA Warner,562
8,9,V Kohli,969


### Top 10 wicket takers overall with their country

In [ ]:
q = """
select count(kind_out) as wickets, p.player_name, c.country_name from
ball_by_ball a
join wicket_taken b
on a.match_id = b.match_id
and a.over_id = b.over_id
and a.ball_id = b.ball_id
and a.innings_no = b.innings_no
join player p
on p.player_id = bowler
join country c
on p.country_name = c.country_id
group by p.player_name
order by wickets desc
limit 10;
"""
print_question(3)
query_execute(q)

Top 10 wicket takers overall with their country


,wickets,Player_Name,Country_Name
0,159,SL Malinga,Sri Lanka
1,137,DJ Bravo,West Indies
2,132,A Mishra,India
3,128,Harbhajan Singh,India
4,127,PP Chawla,India
5,123,R Vinay Kumar,India
6,111,A Nehra,India
7,110,R Ashwin,India
8,107,Z Khan,India
9,100,RP Singh,India


### Display players with top 5 strike rates

In [ ]:
q = """
select d.player_name, strike_rate from
(
  select striker,
  100*round(round(sum(coalesce(Runs_Scored,0)),2)/round(count(*),2),4) as strike_rate from
  ball_by_ball a
  join batsman_scored b
  on a.match_id = b.match_id
  and a.innings_No=b.innings_No
  and a.over_id=b.over_id
  and a.ball_id=b.ball_id
  left join Extra_Runs c
  on a.Match_Id=c.Match_Id
  and a.Innings_No=c.Innings_No
  and a.Over_Id=c.Over_Id
  and a.Ball_Id=c.Ball_Id
  where Extra_Type_Id !=2 or Extra_Type_Id is null
  group by striker
) a
join (select player_id,player_name
from player) d
on a.striker= d.player_id
order by strike_rate desc
limit 10;

"""
print_question(4)
query_execute(q)


Display players with top 5 strike rates 


,player_name,strike_rate
0,CR Brathwaite,218.42
1,Umar Gul,205.26
2,RS Sodhi,200.00
3,KH Pandya,191.13
4,BCJ Cutting,182.50
5,LJ Wright,176.67
6,Shahid Afridi,176.09
7,I Malhotra,175.00
8,SN Khan,173.53
9,AD Russell,173.41


### Man of Series, Orange Cap, Purple Cap winners

In [ ]:
q = """
select season_year, pm.player_name as man_of_the_series, po.player_name as orange_cap, pp.player_name as purple_cap from
season s
join player pm
on s.man_of_the_series = pm.player_id
join player po
on s.orange_cap = po.player_id
join player pp
on s.purple_cap = pp.player_id
"""
print_question(5)
query_execute(q)

Man of Series, Orange Cap, Purple Cap winners


,Season_Year,man_of_the_series,orange_cap,purple_cap
0,2008,SR Watson,SE Marsh,Sohail Tanvir
1,2009,AC Gilchrist,ML Hayden,RP Singh
2,2010,SR Tendulkar,SR Tendulkar,PP Ojha
3,2011,CH Gayle,CH Gayle,SL Malinga
4,2012,SP Narine,CH Gayle,M Morkel
5,2013,SR Watson,MEK Hussey,DJ Bravo
6,2014,GJ Maxwell,RV Uthappa,MM Sharma
7,2015,AD Russell,DA Warner,DJ Bravo
8,2016,V Kohli,V Kohli,B Kumar


### Players with most centuries along with their season

In [ ]:
q = """
select  d.player_name, count(CASE when d.run_in_match >=100 then 1 else null end) as hundreds, d.season from
(
 select c.season_id as season, c.match_id, c.striker as striker,p.player_name, sum(c.runs_scored) as run_in_match from

(
select m.season_id as season_id, m.match_id as match_id, a.striker as striker, b.runs_scored as runs_scored from
ball_by_ball a
join batsman_scored b
on a.match_id = b.match_id
and a.over_id = b.over_id
and a.ball_id = b.ball_id
and a.innings_no = b.innings_no
join match m
on m.match_id = b.match_id
order by 1, 2, 3
)c
join player p
on p.player_id = striker
group by 1,2,3
order by 5 desc
) d
group by player_name
order by hundreds desc
limit 10;
"""
print_question(6)
query_execute(q)


Players with most centuries along with their season


,player_name,hundreds,season
0,CH Gayle,5,6
1,V Kohli,4,9
2,AB de Villiers,3,8
3,V Sehwag,2,7
4,SR Watson,2,8
5,M Vijay,2,3
6,DA Warner,2,5
7,BB McCullum,2,1
8,AC Gilchrist,2,1
9,YK Pathan,1,3



### Types of wickets and total counts

In [ ]:
q = """
select o.out_name, count(kind_out) as out_count from
wicket_taken w
join out_type o
on w.kind_out = o.out_id
group by kind_out
"""
print_question(7)
query_execute(q)

Types of wickets and total counts


,Out_Name,out_count
0,caught,3954
1,bowled,1251
2,run out,697
3,lbw,399
4,retired hurt,8
5,stumped,222
6,caught and bowled,187
7,hit wicket,8
8,obstructing the field,1


In [ ]:

### Player with highest number of sixes and batting hand

Player with highest number of sixes and batting hand


In [ ]:
q = """
select p.player_name,
count(case when b.runs_scored = 6 then 1 else null end) as sixes from
ball_by_ball a
join batsman_scored b
on a.match_id = b.match_id
and a.over_id = b.over_id
and a.ball_id = b.ball_id
and a.innings_no = b.innings_no
join player p
on p.player_id = a.striker
group by a.striker
order by sixes desc
limit 10
"""
print_question(8)
query_execute(q)

Player with highest number of sixes


,Player_Name,sixes
0,CH Gayle,252
1,RG Sharma,164
2,SK Raina,161
3,V Kohli,148
4,YK Pathan,143
5,AB de Villiers,142
6,MS Dhoni,140
7,DA Warner,134
8,Yuvraj Singh,133
9,KA Pollard,125


### Players with most number of ducks

In [ ]:
q = """
select p.player_name, count(runs) as duck_count from
(select Striker,sum(Runs_Scored) as runs
from (select Match_Id,Innings_No,Over_Id,Ball_Id,Striker
      from Ball_by_Ball
      where Striker in (select Player_Out
                      from Wicket_Taken))a
join Batsman_Scored b
on a.Match_Id=b.Match_Id
and a.Innings_No=b.Innings_No
and a.Over_Id=b.Over_Id
and a.Ball_Id=b.Ball_Id
group by Striker,a.Match_Id
having runs==0 ) c
join player p
on p.player_id = c.striker
group by p.player_name
order by duck_count desc
limit 10;

"""
print_question(9)
query_execute(q)

Players with most number of ducks


,Player_Name,duck_count
0,Harbhajan Singh,11
1,G Gambhir,11
2,R Sharma,10
3,PP Chawla,10
4,PA Patel,10
5,MK Pandey,10
6,P Kumar,9
7,NV Ojha,9
8,KD Karthik,8
9,JH Kallis,8


### Select the best Economy bowlers in IPL

In [ ]:
# In cricket, a bowler's economy rate is the average number of runs they have conceded per over bowled.

q = """
select p.player_name,c.country_name, a.overs, a.runs, a.extras, a.economy from
(
  select bowler, sum(coalesce(extra_runs, 0)+ runs_scored) as runs,
  sum(coalesce(extra_runs,0)) as extras,
  count(*)/6 as overs,
  round(round(sum(coalesce(extra_runs, 0) + runs_scored), 2)/round(count(*)/6,2),2) as economy
  from batsman_scored a
  join ball_by_ball b
  on a.match_id = b.match_id
  and a.innings_no = b.innings_no
  and a.over_id = b.over_id
  and a.ball_id = b.ball_id
  left join extra_runs c
  on a.match_id = c.match_id
  and a.innings_no = c.innings_no
  and a.over_id = c.over_id
  and a.ball_id = c.ball_id
  group by bowler
  having overs>=50
) a
join player p
on a.bowler = p.player_id
join country c
on p.country_name = c.country_id
order by economy asc
limit 10;
"""
print_question(10)
query_execute(q)


Select the best economy bowlers in IPL


,Player_Name,Country_Name,overs,runs,extras,economy
0,SP Narine,West Indies,263,1657,52,6.30
1,R Ashwin,India,383,2475,66,6.46
2,SL Malinga,Sri Lanka,386,2534,127,6.56
3,DW Steyn,South Africa,347,2287,78,6.59
4,A Kumble,India,161,1063,36,6.60
5,M Muralitharan,Sri Lanka,256,1709,67,6.68
6,GD McGrath,Australia,54,362,10,6.70
7,Mustafizur Rahman,Bangladesh,61,417,6,6.84
8,DL Vettori,New Zealand,129,888,18,6.88
9,RE van der Merwe,South Africa,73,503,17,6.89


### Indian Players below 21 years of age in the year 2016

In [ ]:
q = """
select p.player_name,c.country_name,t.team_name, 2016-strftime('%Y', p.DOB) as age from
(
select match_id,team_1, season_year from
match m
join season s
on m.season_id = s.season_id
where season_year = '2016'
) a
join ball_by_ball b
on b.match_id = a.match_id
join player p
on p.player_id = b.striker
join country c
on c.country_id = p.country_name
join team t
on t.team_id = a.team_1
group by b.striker
having age < 21
order by age;
"""
print_question(11)
query_execute(q)


Indian players below 21 years of age in the year 2016


,Player_Name,Country_Name,Team_Name,age
0,Ishan Kishan,India,Kings XI Punjab,18
1,SN Khan,India,Royal Challengers Bangalore,19
2,RR Pant,India,Delhi Daredevils,19


### References
* https://mode.com/sql-tutorial/
* https://www.kaggle.com/datasets/harsha547/ipldatabase
* https://www.cricindeed.com/  
* https://knoema.com/insights/cricket
* https://www.kaggle.com/code/sudhanshu2198/ipl-sports-analysis-using-sql